In [ ]:
# Import necessary libraries
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Upload the Excel file from local system
uploaded = files.upload()

Saving Stocks_integ.xlsx to Stocks_integ.xlsx


In [ ]:
# Load the Excel file
file_path = list(uploaded.keys())[0]  # Automatically get the uploaded file name
excel_data = pd.ExcelFile(file_path)

In [ ]:
# Prepare lists to collect P/E, EV/EBITDA ratios and Quarterly Returns for each stock
pe_ratios = []
ev_ebitda_ratios = []
qtr_returns = []
tickers = []

# Loop through each sheet (assuming each sheet name is formatted as 'ticker-V')
for sheet_name in excel_data.sheet_names:
    stock_data = excel_data.parse(sheet_name)

    # Extract P/E, EV/EBITDA and quarterly return values
    pe_ratio = stock_data[stock_data['name'] == 'PeRatio'].iloc[0, 45]
    ev_ebitda_ratio = stock_data[stock_data['name'] == 'EnterprisesValueEBITDARatio'].iloc[0, 45]
    quarterly_returns = stock_data[stock_data['name'] == 'QuarterlyReturns'].iloc[0, 33]

    # Store results
    pe_ratios.append(pe_ratio)
    ev_ebitda_ratios.append(ev_ebitda_ratio)
    qtr_returns.append(quarterly_returns)
    tickers.append(sheet_name)  # Store ticker for reference

# Create a DataFrame with P/E, EV/EBITDA ratios and quarterly returns for all stocks
value_ratios = pd.DataFrame({
    'Ticker': tickers,
    'P/E': pd.to_numeric(pe_ratios, errors='coerce'),
    'EV/EBITDA': pd.to_numeric(ev_ebitda_ratios, errors='coerce'),
    'QuarterlyReturns': pd.to_numeric(qtr_returns, errors='coerce')
})

In [ ]:
# Rank stocks into quintiles for both P/E and EV/EBITDA
value_ratios['P/E Quintile'] = pd.qcut(value_ratios['P/E'], 5, labels=False) + 1
value_ratios['EV/EBITDA Quintile'] = pd.qcut(value_ratios['EV/EBITDA'], 5, labels=False) + 1

In [ ]:
# Quarterly Returns as calculated using average quarter stock price
value_ratios['returns'] = pd.qcut(value_ratios['QuarterlyReturns'], 5, labels=False) + 1  # Assign returns

In [ ]:
# Performance calculations for each quintile (example: P/E quintiles)
pe_performance = value_ratios.groupby('P/E Quintile')['returns'].mean()
ev_ebitda_performance = value_ratios.groupby('EV/EBITDA Quintile')['returns'].mean()

In [ ]:
# Calculate Sharpe ratios for each P/E and EV/EBITDA quintile
risk_free_rate = 0.002  # Assume a risk-free rate, e.g., 0.2% for demonstration
value_ratios['excess_returns'] = value_ratios['returns'] - risk_free_rate

# Sharpe Ratio Calculation for P/E and EV/EBITDA quintiles
pe_sharpe_ratios = value_ratios.groupby('P/E Quintile')['excess_returns'].apply(lambda x: x.mean() / x.std())
ev_ebitda_sharpe_ratios = value_ratios.groupby('EV/EBITDA Quintile')['excess_returns'].apply(lambda x: x.mean() / x.std())

# Display stocks in each quintile for both P/E and EV/EBITDA
# Group stocks by P/E quintile
pe_quintiles = value_ratios.groupby('P/E Quintile')[['Ticker', 'P/E']].apply(lambda x: x.reset_index(drop=True))

# Group stocks by EV/EBITDA quintile
ev_ebitda_quintiles = value_ratios.groupby('EV/EBITDA Quintile')[['Ticker', 'EV/EBITDA']].apply(lambda x: x.reset_index(drop=True))

# Display performance metrics and stocks in each quintile
print("Average Returns for each P/E Quintile:")
print(pe_performance)

print("\nSharpe Ratios for each P/E Quintile:")
print(pe_sharpe_ratios)

print("\nStocks in Each P/E Quintile:")
for quintile, group in pe_quintiles.groupby(level=0):
    print(f"\nP/E Quintile {quintile}")
    print(group)

print("\nAverage Returns for each EV/EBITDA Quintile:")
print(ev_ebitda_performance)

print("\nSharpe Ratios for each EV/EBITDA Quintile:")
print(ev_ebitda_sharpe_ratios)

print("\nStocks in Each EV/EBITDA Quintile:")
for quintile, group in ev_ebitda_quintiles.groupby(level=0):
    print(f"\nEV/EBITDA Quintile {quintile}")
    print(group)

Average Returns for each P/E Quintile:
P/E Quintile
1.0    2.750000
2.0    3.333333
3.0    3.500000
4.0    4.333333
5.0    1.500000
Name: returns, dtype: float64

Sharpe Ratios for each P/E Quintile:
P/E Quintile
1.0    1.332976
2.0    1.600321
3.0    6.058714
4.0    3.751045
5.0    1.498000
Name: excess_returns, dtype: float64

Stocks in Each P/E Quintile:

P/E Quintile 1.0
                Ticker     P/E
P/E Quintile                  
1.0          0    ED-V  16.333
             1   AEP-V  17.561
             2  AAPL-V   9.468
             3  NVDA-V  15.522

P/E Quintile 2.0
                Ticker     P/E
P/E Quintile                  
2.0          0   GIS-V  17.677
             1   PEP-V  18.750
             2  MSFT-V  17.807

P/E Quintile 3.0
                Ticker     P/E
P/E Quintile                  
3.0          0    SO-V  18.859
             1   DUK-V  20.769
             2    PG-V  19.583
             3  BIDU-V  19.187

P/E Quintile 4.0
               Ticker     P/E
P/E Quintil